빅카인즈
https://www.bigkinds.or.kr/


# 국회의원 데이터 스크래핑 

## Selenium 및 웹 드라이버 설치 

In [1]:
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-brower/chromdriver /usr/bin

# import sys
# sys.path.insert(0, '/usr/lib/chromium-brower/chromdriver')

In [2]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
import os
import shutil
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from urllib.request import urlretrieve


## `get_politician_info()`: 국회의원 정보 스크래핑

In [4]:
def get_politician_info(dir_name, wd):
    
    profile_info = wd.find_element_by_xpath('//*[@id="contents"]/div[1]/section[1]')
    
    cols = profile_info.find_elements_by_tag_name('dt')
    columns = [col.text for col in cols]
    
    infos = profile_info.find_elements_by_tag_name('dd')
    detail_info = [info.text for info in infos]
    
    info_dic = {col:info for col,info in zip(columns, detail_info)}
    
    return info_dic

## `get_news_quote()` : 국회의원 뉴스 인용문

In [5]:
def get_news_quote(dir_name, name, wd):
    page_no = 0 
    news_count= 0 
    news_max = 20
    news_total = wd.find_element_by_id('newsInQoutTotalCount').text[1:-2]
    news_file = open(dir_name + '/' + name + '뉴스 인용문.txt', 'w')
    news_max = min(news_max, int(news_total))
    while True:
        try:
            page_no += 1
            box_list = wd.find_element_by_id('depthAnal-quotations-results')
            
            
            quotes = box_list.find_elements_by_class_name('title')
            
            for quote in quotes:
                news_file.write(quote.text + '\n')
                news_count += 1
                print(f" {news_count}/{news_total}", quote.text[:10])
                
         
            if news_count >= news_max:
                
                break
                
            wd.find_element_by_xpath('//*[@id="newsInQuotListPaging"]/a[3]').click()
            
            time.sleep(1)
            
            
        except:
            print('error')
            break
            
        if news_count >= news_max:
            
            break
            
        
    news_file.close()

## `scraping()` : 스크래핑 함수

In [6]:
def scraping(dir_name, politician_max=5):
    page_no = 0 
    politician_count = 0 
    politician_df = pd.DataFrame()
    
    wd = webdriver.Chrome('c:/WebDriver/bin/chromedriver.exe')
    wd.execute_script('window.open("about:blank","_blank");') # 탭 창을 여는 자바 명령어
    tabs = wd.window_handles
    
    while True:
        try:
            
            page_no += 1
            wd.switch_to.window(tabs[0])

            url = f"https://www.bigkinds.or.kr/v2/depthAnalysis/assembly.do?page={page_no}"
            wd.get(url)
            politician_total = wd.find_element_by_xpath('//*[@id="contents"]/section[1]/div/div/div[1]/div[2]').text
            politician_items = wd.find_elements_by_xpath('//*[@id="contents"]/section[3]/div/ul/li')

            if not politician_items:
                break

            for item in politician_items:
                wd.switch_to.window(tabs[0])

                politician_count += 1

                print('-----------------------------------')
                print(f"국회의원{politician_count}/{int(politician_total)}명")
                name = item.find_element_by_class_name('sp-kname').text
                print(name)
                
                detail_link = item.find_element_by_tag_name('a').get_attribute('href')
                
                wd.switch_to.window(tabs[1])
                wd.get(detail_link)
                
                print('[프로필 이미지 다운로드]')
                profile_image = wd.find_element_by_xpath('//*[@id="contents"]/div[1]/div/div/div[2]')
                image_src = profile_image.find_element_by_tag_name('img').get_attribute('src')
                file_name = dir_name + '/' + name + '.jpg'
                
                print(file_name)
                
                urlretrieve(image_src, file_name)
               
                time.sleep(0.1)
                
                print('[기본 정보 스크래핑]')
                politician_dic = get_politician_info(dir_name, wd)
                politician_dic['이름'] = name
                print("  ", politician_dic)
                
                
                politician_df = politician_df.append(politician_dic, ignore_index=True)
                
                print("[뉴스 인용문 스크래핑]")
                
                wd.find_element_by_xpath('//*[@id="tabs-news1"]/div[2]/div[2]/div/ul/li[1]').click
                get_news_quote(dir_name, name, wd)
                
                

                
                
        
                if politician_count>=politician_max:
                    break

                    
            if politician_count>=politician_max:
                    break
                
        except AttributeError as e:
            print(e)
            break
            
        except NoSuchElementException as e:
            print(e)
            break
        
    
        

    wd.close()
    
    return politician_df
    
            

## 스크래핑 시작

In [7]:
dir_name = './politician'
if os.path.isdir(dir_name):
    shutil.rmtree(dir_name)
    
os.makedirs(dir_name)
print(f"{dir_name} 디렉토리 생성")

politician_df = scraping(dir_name)

./politician 디렉토리 생성
-----------------------------------
국회의원1/300명
강기윤(姜起潤)
[프로필 이미지 다운로드]
./politician/강기윤(姜起潤).jpg
[기본 정보 스크래핑]
   {'지역구': '경남 창원시성산구', '당선기록': '재선(19대, 21대)', '소속위원회': '보건복지위원회', '사무실전화': '02-784-1751', '보좌관': '김홍광 , 한영애', '이메일주소': 'ggotop@naver.com', '홈페이지': 'http://blog.naver.com/ggotop', '경력': '[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사 창원대학교 대학원 행정학 박사 [경력] 현) 국회 보건복지위원회 국민의힘 간사 현) 국민의힘 소상공인살리기 특별위원회 부위원장 현) 국민의힘 코로나19 대책 특별위원회 위원 미래통합당 경남도당 민생특위 위원장 제19대 국회의원 (새누리당/경남 창원시 성산구) 새누리당 원내부대표', '이름': '강기윤(姜起潤)'}
[뉴스 인용문 스크래핑]
 1/28 대리수술이나 수술 
 2/28 수술실 내부 CCT
 3/28 내·외부 어디가 좋
 4/28 여야 할 것 없이 
 5/28 대리 수술과 수술실
 6/28 여야 할 것 없이 
 7/28 대리수술이나 성범죄
 8/28 여·야 할 것 없이
 9/28 정부도 수술실 내부
 10/28 대리수술·성범죄·불
 11/28 대리수술이나 성범죄
 12/28 대리수술이나 수술실
 13/28 질병청이 찬성하고 
 14/28 근로자와 기업들을 
 15/28 의혹은 있지만 법적
 16/28 문체부 장관은 삼성
 17/28 문체부 장관은 삼성
 18/28 수술실 출입구에 설
 19/28 공공형어린이집 지정
 20/28 불법체류자의 경우 
-----------------------------------
국회의원2/300명
강대식(姜大植)
[프로필 이미지 다운로드]
./politician/강대식(姜大植).